# Scalar Grad Dev and Demo

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, sys, warnings, os, json
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

## Class Dev

In [16]:
class Scalar(object):
  """
  Stores a single scalar value and its attributes
  """
  
  def __init__(self, data):
    self.data = data
    
  def __add__(self, other):
    return Scalar(self.data + other.data)
  
  def __mul__(self, other):
    return Scalar(self.data * other.data)
  
  def __repr__(self):
    return f"Scalar(data={self.data})"

In [17]:
a = Scalar(2.0)
b = Scalar(3.0)

In [19]:
print(a*b)
print(a+b)

Scalar(data=6.0)
Scalar(data=5.0)


## Derivative

In [ ]:
h = 0.001
x = 3.0